# Bundesdatascrap


Getting NRW's COVID data from : https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100
Based on the tutorials: 
- https://medium.com/opex-analytics/simple-web-scraping-in-python-90d6fddfaeca
- https://medium.com/analytics-vidhya/beginner-to-advance-web-scraping-guide-in-python-799ffd367067
- https://stackoverflow.com/questions/48513576/python-requests-module-doesnt-return-full-page-during-get-request
- https://medium.com/opex-analytics/simple-web-scraping-in-python-90d6fddfaeca


In [30]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [2]:
# Finding URL pattern:

#first 100 cases , it has 23 more
url0 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
url1 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D2&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
url2 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D3&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
url3 = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D4&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'

# Found, creating a base url for pages 1:23 , page 0 must be url0

# creating list of all urls:

results = [url0, url1]

for i in range(2,24):
    i = str(i)    # converting to str
    url_base = 'https://www.service.bund.de/Content/DE/Ausschreibungen/Suche/Formular.html?nn=4641482&gtp=4642258_list%253D' + i + '&type=0&searchResult=true&cl2Addresses_Adresse_State=nordrhein-westfalen&resultsPerPage=100'
    results.append(url_base)
    
#results

# Scrap

Here we begging with scrap. <br>
I saw on mozila firefox that the part of the page that I wanted to scrap what is inside the "ul class "result-list""

![title](html_bundesdatei2.png)

In [3]:
# Scrap

# makes a request to the web page and gets its HTML
r = requests.get(url0,headers={'User-Agent': 'Mozilla/5.0'}) # with the user agent it downloaded all the page

# stores the HTML page in 'soup', a BeautifulSoup object
soup = BeautifulSoup(r.text, 'html.parser')

In [122]:
#looking at what I get inside the ul markers
# print(soup.find_all('ul'))

bs4.element.ResultSet

In [4]:
#Creating an object that will contain all the results inside the marker ul class:"result-list"
content = soup.find('ul', {"class": "result-list"})

content #Uhul! 

<ul class="result-list">
<li>
<a href="IMPORTE/Ausschreibungen/asp/2021/03/125476.html;jsessionid=99F5FD3DF86E429D9F099197955F586D.2_cid376?nn=4641482&amp;type=0&amp;searchResult=true" title="Zur Ausschreibung 'Klaeranlage Kamen-Koernebach, Reinvestition Abwasser- und Schlammbehandlung'"><div aria-labelledby="title">
<h3><em>Ausschreibung</em>Klae­r­an­la­ge Ka­men-Koer­ne­bach, Rein­ves­ti­ti­on Ab­was­ser- und Schlamm­be­hand­lung</h3>
<p><em>Vergabestelle</em> Lippeverband</p>
</div>
<div aria-labelledby="date">
<p><em>Veröffentlicht</em> 11.03.21</p>
</div>
<div aria-labelledby="location">
<p><em>Angebotsfrist</em> 08.06.21</p>
</div>
</a>
</li>
<li>
<a href="IMPORTE/Ausschreibungen/editor/Jobcenter-Oberberg/2021/03/3856386.html;jsessionid=99F5FD3DF86E429D9F099197955F586D.2_cid376?nn=4641482&amp;type=0&amp;searchResult=true" title="Zur Ausschreibung 'Respirationsschutzwand inkl. Lieferung'"><div aria-labelledby="title">
<h3><em>Ausschreibung</em>Re­spi­ra­ti­ons­schutz­wand in­kl. 

More specific, I want the following structure to build my dataframe:
    
![title](html_bundesdatei3.png)

In [71]:
# what I want is insite the marker a, so Im going to get the content and get only the a inside it:
links = content.find_all('a')

# In paralel I'm making a string list to get the html :)
links_list = []

for x in links:
    links_list.append(str(x))

# Validating
#links_list[0]

# Agora eu preciso criar uma tabela lateral que vai receber os htmls para que eu possa rodar o segundo scrap:

links2 = [sub.replace('<a href=', '') for sub in links_list] 
links2 = [sub.replace('"', '') for sub in links2] 
links2 = [re.sub('\s.*', '', sub) for sub in links2]
links2 = ['https://www.service.bund.de/' + sub for sub in links2] 

df_links = pd.DataFrame(links2)
# df_links

In [ ]:
df = pd.DataFrame(links)
df = df.iloc[:, [0,2,4]]


In [69]:
# MAking the fisrt DF
df = pd.DataFrame(links)
df = df.iloc[:, [0,2,4]]
df_final = pd.merge(df, df_links, left_index=True, right_index=True)
df_final = df_final.rename(columns={'0_x': "ausschreibung",
                                    2: "veroeffentlicht",
                                    4:"angebotsfrist",
                                    '0_y' : 'link'})



#df_final.replace(r"\[\\n,\s\[\[Ausschreibung\],\s", "", regex=True)


#Exportando em CSV
df_final.to_csv(r'teste.csv')
df_final
#df_final.dtypes
#df_final['ausschreibung'] = df_final['ausschreibung'].str.replace(r'.*\s', 'AA') 
#df_final = df_final.replace(to_replace ='\[\\n,\s\[\[Ausschreibung\],\s', value = '', regex = True) 

C:\Users\yvfg3118\Anaconda3\lib\site-packages\pandas\core\internals\construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])


,ausschreibung,veroeffentlicht,angebotsfrist,link
0,"[\n, [[Ausschreibung], Klae­r­an­la­ge Ka­men-...","[\n, [[Veröffentlicht], 11.03.21], \n]","[\n, [[Angebotsfrist], 08.06.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
1,"[\n, [[Ausschreibung], Re­spi­ra­ti­ons­schutz...","[\n, [[Veröffentlicht], 11.03.21], \n]","[\n, [[Angebotsfrist], 19.03.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
2,"[\n, [[Ausschreibung], Up­gra­de und Neu­kauf ...","[\n, [[Veröffentlicht], 11.03.21], \n]","[\n, [[Angebotsfrist], 18.03.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
3,"[\n, [[Ausschreibung], Ent­wick­lungs­pfle­ge ...","[\n, [[Veröffentlicht], 11.03.21], \n]","[\n, [[Angebotsfrist], 30.03.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
4,"[\n, [[Ausschreibung], Lie­fe­rung „Bahn­über­...","[\n, [[Veröffentlicht], 11.03.21], \n]","[\n, [[Angebotsfrist], 30.03.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
...,...,...,...,...
95,"[\n, [[Ausschreibung], Neu­bau Carl Fried­rich...","[\n, [[Veröffentlicht], 10.03.21], \n]","[\n, [[Angebotsfrist], 08.04.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
96,"[\n, [[Ausschreibung], IT-Dienst­leis­tung An­...","[\n, [[Veröffentlicht], 10.03.21], \n]","[\n, [[Angebotsfrist], 13.04.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
97,"[\n, [[Ausschreibung], B236, ÜF über L743, Er­...","[\n, [[Veröffentlicht], 10.03.21], \n]","[\n, [[Angebotsfrist], 13.04.21], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
98,"[\n, [[Ausschreibung], Sicht­prü­fung und Do­k...","[\n, [[Veröffentlicht], 10.03.21], \n]","[\n, [[Angebotsfrist], ], \n]",https://www.service.bund.de/IMPORTE/Ausschreib...
